In [6]:
# set crs
crs= {'init': 'epsg:6511'}

In [1]:
import pandas
import geopandas
from path import Path
import os
import sys
import re
from functools import reduce

In [2]:
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon

In [3]:
# path configuration
working_directory = Path.getcwd()

inputs_path = working_directory / 'inputs'
outputs_path = working_directory / 'outputs'

In [4]:
# export function
def to_file(callback, desired_name, extension='csv'):
    current_files = sorted(outputs_path.files(desired_name + '*.' + extension))
    if current_files:
        last_file = current_files[-1]
        os.remove(last_file)
    final_name = '{}.{}'.format(desired_name, extension)
    callback(outputs_path / final_name)

In [9]:
# read in zone_points and get polygon 
%store -r zone_points
mo_places = geopandas.read_file(inputs_path / 'tl_2017_29_place/tl_2017_29_place.shp').to_crs(6511)

In [ ]:
zone_points.groupby(hwy_cols).count()

In [ ]:
zone_points[hwy_cols]

{1:'Yes',
2:'No'}

In [ ]:
# filter to areas of interest
mo_places = mo_places[mo_places.NAME.str.contains('Pleasant Hill|Strasburg')]

# view
display(
    mo_places.head(),
    mo_places.plot()
)

In [15]:
# return all points that fall into polygon for assessment
filtered_pnts = zone_points.overlay(mo_places,how='difference',keep_geom_type =True)

In [51]:
# safety measures
hwy_cols = [col for col in zone_points.columns if any(keyword in col for keyword in ['Hwy', 'Highway'])]
trk_cols = [col for col in zone_points.columns if any(keyword in col for keyword in ['Trk', 'Truck','WeekTrnMov'])] 
rr_cols = [col for col in zone_points.columns if any(keyword in col for keyword in ['Sign','Gate','Flash','Illum','Bells','Std','Surface','Xing', 'Railroad'])] 
trf_cols = [col for col in zone_points.columns if any(keyword in col for keyword in ['Spd','Traf','Pass','Thru', 'Aadt'])] 
sfty_cols = [col for col in zone_points.columns if any(keyword in col for keyword in ['Hazmt','Emr'])] 

In [53]:
# fill nana
filtered_pnts[trk_cols] = filtered_pnts[trk_cols].fillna(0)

In [71]:
#Define replacements (dictionary where keys are old values and values are new values)
boolean_replacements = {
    r'\b2\b': 'N', # replacements are regex so that the begininng/end of a string isn't impacted
    r'\b1\b': 'Y'
}

# Perform str.replace() for each column in col_list
for col in hwy_cols:
    filtered_pnts[hwy_cols] = filtered_pnts[hwy_cols].replace(replacements, regex=True)

In [37]:
df.TypeXing.unique() # pub priv
StopStd, YieldStd, Bells # unique number
Illumina, no illum

array(['2', '3'], dtype=object)

In [ ]:
# zone points - 0/2

In [44]:
zone_points.FlashNov.unique()

array(['0'], dtype=object)

In [ ]:
pandas.DataFrame(zone_points[rr_cols])

In [41]:
zone_points.Illumina.unique()

array([None, '2', ''], dtype=object)

In [ ]:
zone_points[rr_cols]

In [ ]:
fig, ax = plt.subplots()

mo_places.plot(ax=ax, color='white', edgecolor='black')
filtered_pnts.plot(ax=ax, marker='o', color='red', markersize=5)

plt.show();

In [ ]:
# review data inventory
pandas.DataFrame(filtered_pnts.groupby(hwy_cols).size())
pandas.DataFrame(filtered_pnts.groupby(trk_cols).size())
pandas.DataFrame(filtered_pnts.groupby(rr_cols).size())
pandas.DataFrame(filtered_pnts.groupby(trf_cols).size())
pandas.DataFrame(filtered_pnts.groupby(sfty_cols).size())

### Store and export

In [22]:
%store -r df
%store mo_places

In [ ]:
# export filtered points as shpe file
callback = lambda p: filtered_pnts.to_file(p, driver='ESRI Shapefile')
to_file(callback, 'MoDOT_XX_Amtrak_Route_Filtered', 'shp')

In [ ]:
dfs = [df, pandas.DataFrame(filtered_pnts)]

with pandas.ExcelWriter('MO_Crossing_Points_Raw_Filtered.xlsx') as writer:  # Specify the Excel file name
    for i, df in enumerate(dfs):
        # Use the DataFrame name as the sheet name (you may need to adjust this part)
        sheet_name = f'Sheet_{i}'  # You can customize the sheet names
        df.to_excel(writer, sheet_name=sheet_name, index=False)

# Save the Excel file
writer.close()